# The Non Random Two Liquids (NRTL) model for *excess Gibbs energy* ($g^E$) and a case study of Liquid-Liquid equilibria of water+ethanol+ethyl acetate

## Fitted parameters
Renon et al. (1969) fitted 1 $\alpha$ valid for all 3 binary interactions $\{(1,2),(1,3),(2,3)\}$ parameter and 6 $A_{i,j}$ parameters, two for each binary interaction filling a non symmetric $A$ matrix.

In [2]:
#unit system constant
R=8.314

# Ethyl acetate (1) + water (2) + ethanol (3)

alpha12 = 0.4

alpha23 = 0.3

alpha13 = 0.3

# 6 binary Aij parameters
Dg12 = 1335 #K
Dg21 = 2510 #K

Dg23 = 976 #K
Dg32 = 88 #K

Dg13 = 301 #K
Dg31 = 322 #K


## Feeding the fitted parameters to the model in matrix structure:
we will assemble the parameters in a matrix structure so that we can access each parameter by its index, as in
`A[0,0]` and `A[0,1]`rather than as `A11` and `A12`, so we can loop trough all of them using an iterator, see below:

In [5]:
import numpy as np
from scipy.constants import R

In [11]:
#assemble matrix with regressed parameters Dg_i,j, according to the model all diagonal terms are zero
Dg = np.array([[0, Dg12, Dg13],
             [Dg21, 0, Dg23],
             [Dg31, Dg32, 0]])


#assemble symmetric matrix alpha
alpha = np.array([[0, alpha12, alpha13],
                [alpha12, 0, alpha23],
                [alpha13, alpha23, 0]])

#finally
A = Dg/R


In [74]:
# chafer's

alpha12=alpha13=alpha23=alpha21=alpha31=alpha23=0.2;


A12=107.99
A21=555.81
A13=1011.98
A31=2277.37
A23=-1113.1
A32=1217.37

alpha = np.array([[0, alpha12, alpha13],
                [alpha12, 0, alpha23],
                [alpha13, alpha23, 0]])

A = np.array([[0, A12, A13],
             [A21, 0, A23],
             [A31, A32, 0]])

In [68]:
from numba import jit
@jit
def Gamma(T,c_x,q_alpha, q_A):
    #note that we used many lines for didatics
    #we can do it in few lines:
    #note that some expression occur more than once below
    #so it may be useful define it as a intermediary recurrent term here
    #and calculate it once to use it then several times
    q_tau     = q_A/T
    q_G       = np.exp(-(q_alpha*q_tau))
    l_D       = ((1/((q_G.T) @ c_x)).T)
    q_E       = (q_tau*q_G) * l_D 
    gamma     = np.exp(((q_E+(q_E.T))-(((q_G * l_D) * (c_x.T)) @ (q_E.T))) @ c_x)
    return gamma

In [69]:
#test it to see that the results are the same
T=293.15
x=np.array([.2,.3,.5]) #normalized
x_as_column = np.array([x]).T
print(Gamma(T,x_as_column,alpha,A)) #test using those trial input

[[ 7.97865398]
 [ 0.31550702]
 [ 0.95773617]]


# Liq-Liq Equilibria Flash
The next step will be using the model in a phase equilibria algorithm, a Liq-Liq equilibria flash calculation

it works as follows:

The degrees of Freedom for a flash calculation are temperature - T, pressure - P and global composition - z


XXX img src from thesis XXX
* ...............  =>guess, xL1 xL2, then calls model(T,P,x)
* ...............//
* .............<=
* T,P,z => algorithm => finds equilibrium xL1 xL2 and BETA

**Equilibriuma criteria**

$$\mu_{i}^{\rm {L1}}=\mu_{i}^{\rm {L2}}, \forall i$$

**Devised algorithm after analytical simplification of repeated contributions:**

$$x_{i}^{\rm {L1}} \gamma_{i}^{\rm {L1}}= x_{i}^{\rm {L2}} \gamma_{i}^{\rm {L2}}, \forall i$$

# algorithm


In [70]:
def ELLflash(Z,beta0,K0,MODEL):
    v1 = np.array([[1,1,1]]).T
    beta=prevbeta=beta0
    K=prevK=K0

    G=np.exp(-alpha*A/T);

    Lambda=(A*G/T);
    
    looped2 = 0
    looped3 = 0

    #Gerencia Loop até que atinja convergência
    looping2 = 0
    while (looping2 == 0) or (max(abs(K-prevK))>0.0001 and looping2 < 1000):
    #while max(abs(K-prevK))>0.0001 && looping2 < 1000
        prevbeta = beta
        a= (1./(v1+beta*(K-v1))) * (K-v1)
        beta = beta+(1/((Z.T*a.T) @ a)) @ (Z.T @ a)

        looping3 = 0;
        while (abs(beta-prevbeta)>0.0001 and looping3 < 1000):
            prevbeta = beta
            a=(1./(v1+beta*(K-v1))) * (K-v1)
            beta=beta+(1/((Z.T * a.T) @ a)) @ (Z.T @ a)
            looping3 = looping3 + 1
            looped3 = looped3 + 1
        

        prevK = K
        XI=(1./(v1+beta * (K-v1))) * Z
        EI=Lambda * (1./(G.T @ XI)).T
        LI=G * (1./(G.T @ XI)).T
        gamaI=np.exp((EI+EI.T-LI * XI.T @ EI.T) @ XI)
        XII=K*XI;
        EII=Lambda* (1./(G.T @ XII)).T
        LII=G*(1./(G.T @ XII)).T
        gamaII=np.exp((EII+EII.T-LII*XII.T @ EII.T) @ XII); #o modelo do gamma tá embutido
        K=np.exp(np.log(gamaI)-np.log(gamaII));
                       
        looping2 = looping2 + 1;
        looped2 = looped2 + 1;
    

    print('Calculation has ended')
    if (looped2 < 1000 and looped3 < 1000):
        Converged = 1;
        print('xi',XI)
        print('xii',XII)
        print('beta',beta)
    else:
        Converged = 0;
        print('unconverged')
    
    return XI, XII, beta

In [79]:
ztest=np.array([[0.3000,
0.4000,
0.3000]]).T

betatest=0.2966

xitest=np.array([[0.0535,
0.5250,
0.4215]]).T


xiitest=np.array([[0.8845,
0.1035,
0.0120]]).T

ktest=1/(xitest/xiitest)

beta0=betatest*1
k0=ktest*1
Z=ztest

print(T)
print(xitest*Gamma(T,xitest,alpha,A))
print(xiitest*Gamma(T,xiitest,alpha,A))

print('hue')
print(ELLflash(Z,beta0,k0,0))

293.15
[[ 0.9168827 ]
 [ 0.3519015 ]
 [ 0.17377495]]
[[ 0.91689205]
 [ 0.35194092]
 [ 0.17381633]]
hue
Calculation has ended
xi [[ 0.05351624]
 [ 0.52503406]
 [ 0.4214497 ]]
xii [[ 0.88451339]
 [ 0.10349332]
 [ 0.01199329]]
beta [[ 0.29661204]]
(array([[ 0.05351624],
       [ 0.52503406],
       [ 0.4214497 ]]), array([[ 0.88451339],
       [ 0.10349332],
       [ 0.01199329]]), array([[ 0.29661204]]))


# initial guess

%Estimativas Iniciais
X0=Z;
E0=Lambda*diag(1./(G'*X0));
L0=G*diag(1./(G'*X0));
gama0=exp([E0+E0'-L0*diag(X0)*E0']*X0);

Xlin=1/(gama0'*Z)*diag(gama0)*Z;
Elin=Lambda*diag(1./(G'*Xlin));
Llin=G*diag(1./(G'*Xlin));
gamalin=exp([Elin+Elin'-Llin*diag(Xlin)*Elin']*Xlin);
Klin=diag(1./gamalin)*gama0;
KlinMin=min(Klin);
KlinMax=max(Klin);

nI0=diag(KlinMax*v1-Klin)*Z;
XI0=1/(v1'*nI0)*nI0;
EI0=Lambda*diag(1./(G'*XI0));
LI0=G*diag(1./(G'*XI0));
gamaI0=exp([EI0+EI0'-LI0*diag(XI0)*EI0']*XI0);

nII0=diag(Klin-KlinMin*v1)*Z;
XII0=1/(v1'*nII0)*nII0;
EII0=Lambda*diag(1./(G'*XII0));
LII0=G*diag(1./(G'*XII0));
gamaII0=exp([EII0+EII0'-LII0*diag(XII0)*EII0']*XII0);

beta0=1/(KlinMax-KlinMin)*v1'*nII0;
K0=exp(log(gamaI0)-log(gamaII0));


# grid calc plot

In [ ]:
Resultados_beta = zeros(1,100);
Resultados_Z = zeros(3,100);
Resultados_XI = zeros(3,100);
Resultados_XII = zeros(3,100);
j = 0;
for z1 = [0:0.1:1]
	for z2 = [0:0.1:1]
        if z1+z2 <= 1
        roda_nrtl_iuri
        if Converged == 1 && beta > 0 && beta < 1;
        j = j+1;
        Resultados_beta(j) = beta;
            for l = [1;2;3]
                Resultados_XI(l,j) = XI(l);
                Resultados_XII(l,j) = XII(l);
                Resultados_Z(l,j) = Z(l);
            end %123
        end %converged
        end %<=1
    end %z2
end %z1